In [7]:
import numpy as np
import time
import cv2
import os
import random
import sys

CATEGORIES = ["2", "4", "5", "6", "9"]
DATADIR = "C:/Users/Enzo/Dataset/mnist_png/training"
IMG_SIZE = 28
TESTDIR = "C:/Users/Enzo/Dataset/mnist_png/testing"

class NeuralNetwork():
    def __init__(self, sizes, epochs=10, lr=0.001):
        self.sizes = sizes
        self.epochs = epochs
        self.lr = lr
        self.params = {
            'W1': np.random.randn(sizes[1], sizes[0]),
            'W2': np.random.randn(sizes[2], sizes[1]),
            'W3': np.random.randn(sizes[3], sizes[2]),
            'B1': np.random.randn(sizes[1],),
            'B2': np.random.randn(sizes[2],),
            'B3': np.random.randn(sizes[3],),
        }

    def sigmoid(self, x, deri=False):
        if deri:
            return self.sigmoid(x) * (1 - self.sigmoid(x))
        return 1 / (1 + np.exp(-x))

    def softmax(self, x, deri=False):
        if deri:
            return self.softmax(x) * (1 - self.softmax(x))
        return np.exp(x) / np.sum(np.exp(x), axis=0)
    
    def relu(self, x, deri=False):
        if deri:
            return 1 * (x > 0)
        return x * (x > 0)

    def get_accu(self, x_val, y_val):
        predictions = []
        for x, y in zip(x_val, y_val):
            output = self.feedforward(x)
            pred = int(CATEGORIES[np.argmax(output)])
            predictions.append(pred == int(CATEGORIES[np.argmax(y)]))
        return np.mean(predictions)

    def train(self, x_train, y_train, x_val, y_val):
        start_time = time.time()
        count = 0
        for iteration in range(self.epochs):
            for x,y in zip(x_train, y_train):
                count += 1
                output = self.feedforward(x)
                changes_to_w = self.backward_prog(y, output)
                self.update_network_parameters(changes_to_w)
                progress_bar(count, 20000, round(time.time() - start_time, 2))
            accuracy = self.get_accu(x_val, y_val)
            print('\nEpoch: {0}, Time Spent: {1:.2f}s, Accuracy: {2:.2f}%'.format(iteration+1, time.time() - start_time, accuracy * 100))
            count = 0
        
    def predict(self, x_test):
        predictions = []
        for x in x_test:
            output = self.feedforward(x)
            pred = int(CATEGORIES[np.argmax(output)])
            predictions.append(pred)
        return predictions
    
    def test(self, x_test, y_test):
        predictions = []
        for x, y in zip(x_test, y_test):
            output = self.feedforward(x)
            pred = int(CATEGORIES[np.argmax(output)])
            predictions.append(pred == int(CATEGORIES[np.argmax(y)]))
        print('Test Accuracy: {0:.2f}%'.format(np.mean(predictions) * 100))
        return np.mean(predictions)
    
    def feedforward(self, x_train):
        params = self.params
        params['A0'] = x_train
        params['Z1'] = np.dot(params['W1'], params['A0']) + params['B1']
        params['A1'] = self.sigmoid(params['Z1'])
        params['Z2'] = np.dot(params['W2'], params['A1']) + params['B2']
        params['A2'] = self.sigmoid(params['Z2'])
        params['Z3'] = np.dot(params['W3'], params['A2']) + params['B3']
        params['A3'] = self.softmax(params['Z3'])
        return params['A3']
    
    def backward_prog(self, y_train, output):
        params = self.params
        weight_change = {
            'W1': np.zeros(params['W1'].shape),
            'W2': np.zeros(params['W2'].shape),
            'W3': np.zeros(params['W3'].shape),
        }
        # Calculate the error
        error = output - y_train
        # Calculate the change in the output layer
        delta_output = error * self.softmax(params['Z3'], deri=True)
        # Calculate the change in the hidden layer
        delta_hidden2 = np.dot(params['W3'].T, delta_output) * self.sigmoid(params['Z2'], deri=True)
        delta_hidden1 = np.dot(params['W2'].T, delta_hidden2) * self.sigmoid(params['Z1'], deri=True)
        # Calculate the change in the weights
        weight_change['W3'] = np.outer(delta_output, params['A2'].T)
        weight_change['W2'] = np.outer(delta_hidden2, params['A1'].T)
        weight_change['W1'] = np.outer(delta_hidden1, params['A0'].T)
        return weight_change
    
    def update_network_parameters(self, changes_to_w):
        self.params['W1'] -= self.lr * changes_to_w['W1']
        self.params['W2'] -= self.lr * changes_to_w['W2']
        self.params['W3'] -= self.lr * changes_to_w['W3']

def progress_bar(current, total, time, barLength = 20):
    if current % 20 != 0:
        return
    percent = float(current) * 100 / total
    arrow = '-' * int(percent/100 * barLength - 1) + '>'
    spaces = ' ' * (barLength - len(arrow))
    sys.stdout.write("\rPercent: [{0}] {1:.0f}% {2:.2f}s".format(arrow + spaces, percent, time))
    sys.stdout.flush()

def load_data():
    training_data = []
    testing_data = []
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass

    for category in CATEGORIES:
        path = os.path.join(TESTDIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                testing_data.append([new_array, class_num])
            except Exception as e:
                pass

    return training_data, testing_data

def to_categorical(y, num_classes=None, dtype='float32'):
    y = np.array(y, dtype='int')
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes), dtype=dtype)
    try:
        categorical[np.arange(n), y] = 1
    except IndexError:
        raise ValueError('y contains an invalid label. Labels must be in [0, {0}]'.format(num_classes - 1))
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)
    return categorical

np.random.seed(42)
random.seed(42)
training_data, testing_data = load_data()
# shuffle training data
random.shuffle(training_data)
# split training data into features and labels
x_train = []
y_train = []
for features, label in training_data:
    x_train.append(features)
    y_train.append(label)
# split testing data into features and labels
x_test = []
y_test = []
for features, label in testing_data:
    x_test.append(features)
    y_test.append(label)
# convert to numpy arrays
x_train = np.array(x_train).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_train = np.array(y_train)
x_test = np.array(x_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_test = np.array(y_test)
# normalize data
x_train = (x_train / 255.0).astype('float32')
x_test = (x_test / 255.0).astype('float32')
# reshape to x_train to match [784, 128] shape
x_train = x_train.reshape(IMG_SIZE**2, -1).T
x_test = x_test.reshape(IMG_SIZE**2, -1).T
nn = NeuralNetwork([784, 64, 32, 5], epochs=1, lr=0.001)
nn.train(x_train, y_train, x_test, y_test)
nn.test(x_test, y_test)
    


Percent: [---------------------------->] 145% 26.32s
Epoch: 1, Time Spent: 26.82s, Accuracy: 59.18%
Test Accuracy: 59.18%


0.5918325466858199

In [8]:
# get the data in the directory testing, and put in a txt file '710882199': prediction
import cv2

def test():
    # get file in the testing directory
    files = os.listdir('./Testing_data1')
    # if the fil exists, delete it
    if os.path.exists('result.txt'):
        os.remove('result.txt')
    # create a file to write the result
    f = open('result.txt', 'w')
    data = []
    for file in files:
        # read the image
        img = cv2.imread('./Testing_data1/' + file, cv2.IMREAD_GRAYSCALE)
        # resize the image
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        # reshape the image
        img = img.reshape(IMG_SIZE**2, -1).T
        # normalize the image
        img = (img / 255.0).astype('float32')
        # predict the image
        pred = nn.predict(img)
        # write the result to the file
        f.write(file.split('.')[0] + ' ' + str(pred[0]) + '\n')
    f.close()

test()

    

In [76]:
# compare each line of the result.txt with the 710882199.txt
# score is the percent of line that are the same
def compare():
    f1 = open('result.txt', 'r')
    f2 = open('710882199.txt', 'r')
    lines1 = f1.readlines()
    lines2 = f2.readlines()
    score = 0
    for i in range(len(lines1)):
        if lines1[i] == lines2[i]:
            score += 1
    print('score: ' + str(score / len(lines1) * 100) + '%')

compare()

score: 15.32%
